In [17]:
# !pip install pyarrow
import pandas as pd
import pyarrow.parquet as pq
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [3]:
DATA_DIR = '../../data/'
http_bots = pq.read_table(DATA_DIR + 'botsHTTPRequests-20180216_1416GMT.parquet').to_pandas()
http_users = pq.read_table(DATA_DIR + 'usersHTTPRequests-20180216_1416GMT.parquet').to_pandas()
#http_bots = http_bots.sample(frac = 0.03)
#http_users = http_users.sample(frac=0.03)
http_users['target'] = 0
http_bots['target'] = 1

http = pd.concat([http_bots, http_users])

http.head()

,browserId,from,ip,method,operation,referrer,requestType,timestamp,to,url,userAgent,userId,hour,target
0,8.293748e+18,None,3rRnET9ABG3feVMANINXpg==,GET,spring-mvc,https://www.ok.ru/,REQ,1518791752182,None,https://ok.ru/web-api/pts/postingFormJs,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,0hs35/dB/t+uT7u/c71+Vw==,17,1
1,8.293748e+18,friendMain,3rRnET9ABG3feVMANINXpg==,POST,friendAltGroup,https://www.ok.ru/,NAV,1518793194690,friendAltGroup,https://www.ok.ru/profile/561514539517/groups,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,HW3KxEEODcSX6gJg1YZQtQ==,17,1
2,7.895490e+17,None,Qsv9DECmtHgfUZ0GAVUoLQ==,POST,PinsEditAjaxRB,https://ok.ru/,REQ,1518791913138,None,https://ok.ru/profile/589603942951/statuses,Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537....,e9iJCewg5Ww1ijKmYKdAyg==,17,1
3,8.293748e+18,None,3rRnET9ABG3feVMANINXpg==,GET,spring-mvc,https://www.ok.ru/,REQ,1518792629926,None,https://ok.ru/web-api/pts/webpush,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,HW3KxEEODcSX6gJg1YZQtQ==,17,1
4,-8.322615e+18,None,XK5THOHUVJHpqGojQAKrMQ==,GET,userMain,None,NAV,1518795472623,userMain,https://ok.ru/,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,f3Z8Sr501bB3SvEgCo+9Rg==,18,1


In [4]:
http = http.sample(frac=1).reset_index(drop=True)

In [5]:
http.head()

,browserId,from,ip,method,operation,referrer,requestType,timestamp,to,url,userAgent,userId,hour,target
0,8.010473e+18,None,VgWa6Lxy+HN9XdZ7eE69Qw==,POST,MainFeedsContent,https://www.ok.ru/,REQ,1518789706991,None,https://www.ok.ru/feed,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,GauR9j1eBNS+eYZ6BMA1eA==,17,0
1,8.263001e+18,None,Dj3d6MMnKEUUovNf0S6Lhg==,POST,videoStatNew,https://ok.ru/,REQ,1518792801301,None,https://ok.ru/dk,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,AaiI61J4ktOA4Lg+hTmT6A==,17,0
2,-7.853830e+18,None,aFGp0LKWmdFTS/ajAZs3Uw==,POST,SuggestStickers,https://ok.ru,REQ,1518795783743,None,https://ok.ru/dk,Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47...,3eEbehs9EWOHKfvhKbCdVg==,18,0
3,8.934099e+18,None,Qd1wKhNL0fmlSqc52m6oIA==,POST,SuggestStickers,https://ok.ru/,REQ,1518792975949,None,https://ok.ru/dk,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,ZtxFovEojQ45DcWW6bjxPw==,17,0
4,-1.066853e+18,None,OC/XKgc4t7gT25V/GkVg+g==,POST,SuggestStickers,https://ok.ru/,REQ,1518791371432,None,https://ok.ru/dk,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,vm0dT30bOSj1nwr30UKMGg==,17,0


In [6]:
print('Кол-во уникальных операций: {}'.format(len(http['operation'].unique())))

Кол-во уникальных операций: 480


### Приведем категориальный столбей общей выборки к разряженному виду и разделим выборку на train и test

Сначала закодируем названия операций чиселками

In [7]:
operations = http['operation']

dict_code_op = {}

c = 0
for op in operations.unique():
    dict_code_op[op] = c
    c += 1
    
operations = operations.apply(lambda x: dict_code_op[x])    
#operations = pd.DataFrame()

In [8]:
operations_sparse = csr_matrix(([1] * operations.shape[0],
                                operations,
                                range(0, operations.shape[0] + 1, 1)))[:, 1:]
#operations_sparse.toarray()

In [9]:
# С помощью train_test_split разбивать не получается, 
# тк разряженный категориальный стобец требует индекса
# и не хочется преобразовывать csr_matrix к pd.DataFrame

# X_train, X_holdout, y_train, y_holdout = train_test_split(
#     http.drop('target', axis=1), http['target'], test_size=0.3, random_state=17)

# operations_sparse -> X_train/X_test; http.target -> y_train/y_test
idx_split = int(http.shape[0] * 0.7)
X_train = operations_sparse[:idx_split]
X_holdout = operations_sparse[idx_split:] #X_test = operations_sparse[idx_split:, :] для df
y_train = http['target'][:idx_split]
y_holdout = http['target'][idx_split:]

### Обучим модель

In [10]:
%%time
# Посмотрим вообще как долго обучаться

logit = LogisticRegression(C=0.1, n_jobs=-1, random_state=17)
logit.fit(X_train, y_train)

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


CPU times: user 1.64 s, sys: 47.7 ms, total: 1.68 s
Wall time: 1.77 s


In [11]:
holdout_pred = logit.predict_proba(X_holdout)

In [12]:
holdout_pred.shape

(303114, 2)

In [37]:
holdout_pred[:15, 1]

array([0.00081123, 0.00302708, 0.00156538, 0.00179828, 0.0003549 ,
       0.00179828, 0.00179828, 0.00944967, 0.00179828, 0.00143255,
       0.00179828, 0.00302708, 0.00108724, 0.0003549 , 0.02993953])

In [14]:
n_bots = 0
for i in holdout_pred:
    if i[0] < i[1]:
        n_bots += 1
print('Грубая оценка кол-ва ботов: {} из {}'.format(n_bots, http.shape[0]))

Грубая оценка кол-ва ботов: 539 из 1010377


In [58]:
roc_auc_score(y_holdout, holdout_pred[:, 1])

0.9216548585044153

In [52]:
# P >= 0.5 => 1
res = []
arr = holdout_pred[:, 1]
type(arr)
for i in arr:
    if i >= 0.5: res.append(1)
    else: res.append(0)

In [55]:
print(metrics.classification_report(y_holdout, res))

             precision    recall  f1-score   support

          0       0.99      1.00      1.00    300023
          1       0.74      0.13      0.22      3091

avg / total       0.99      0.99      0.99    303114

